In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, JavascriptException, StaleElementReferenceException

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install())
)

driver.get("https://data.ex.co.kr/portal/fdwn/view?type=ETC&num=77&requestfrom=dataset")

select_values = {
    "deptName" : "3           ",
    "regionName" : "KR-41       ",
    "gender" : "0           ",
    "userAgeName" : "20          ",
    "pouName" : "0           ",
}
wait = WebDriverWait(driver, 30)

# 활용목적 자동 선택
for select_id, value in select_values.items():
    try:
        Select(
            wait.until(
                EC.presence_of_element_located((By.ID, select_id))
            )
        ).select_by_value(value)
    except TimeoutException:
        print(f"{select_id} 요소를 찾을 수 없음")

# 달력 : 달력 열기
try:
    wait.until(
        lambda d: d.execute_script("return window.jQuery != undefined")
    )
except TimeoutException:
    print("❌ jQuery가 로딩되지 않았습니다.")
    driver.quit()
    raise SystemExit

try:
    driver.execute_script("""
        if (window.jQuery && $('#dataSupplyDate').datepicker) {
            $('#dataSupplyDate').datepicker('show');
        } else {
            throw 'datepicker not initialized';
        }
    """)
except JavascriptException as e:
    print("❌ DatePicker 실행 중 JavaScript 오류 발생")
    print(e.msg)
    driver.quit()
    raise SystemExit

# 달력 : 월 선택
try:
    Select(
        wait.until(
            EC.presence_of_element_located((By.ID, "ui_datepicker_monthid"))
        )
    ).select_by_value("0")
except TimeoutException:
    print("❌ ui_datepicker_monthid 요소를 찾을 수 없음")

# 달력 : 년 선택
try:
    Select(
        wait.until(
            EC.presence_of_element_located((By.ID, "ui_datepicker_yearid"))
        )
    ).select_by_value("2020")
except TimeoutException:
    print("❌ ui_datepicker_yearid 요소를 찾을 수 없음")

# 달력 : 일자 선택
try:
    xpath = (
        "//td[@data-handler='selectDay']"
        "/a[normalize-space(text())='1']"
    )
    wait.until(EC.element_to_be_clickable((By.XPATH, xpath))).click()

except TimeoutException:
    print("❌ 날짜 '1'을 클릭할 수 없습니다 (요소 미존재 또는 비활성).")

except StaleElementReferenceException:
    print("❌ 달력이 다시 렌더링되어 요소 참조가 끊어졌습니다.")

# 검색 버튼 클릭
try:
    wait.until(
        EC.element_to_be_clickable((By.ID, "btnSearch"))
    ).click()
except TimeoutException:
    print("❌ btnSearch 요소를 찾을 수 없음")





# 다운로드 버튼 클릭
# try:
#     wait.until(
#         EC.element_to_be_clickable((By.ID, "fileDownload"))
#     ).click()
# except TimeoutException:
#     print("❌ fileDownload 요소를 찾을 수 없음")


